## READ ME FIRST

This notebook produces a weekly release from the `develop` branch of Cytoscape.

The weekly release consists of Install4j generated installers that can be put on a public server to be downloaded by testers. This script will NOT push any code changes or resources to GitHub or Nexus or update any additional resources like the manual or API docs. 

This notebook is configured for the build server on which it is run. Notes have been made where local environment variables have been set.

Before starting this process, it is a good idea to Restart and Clear Output for this notebook's Kernel.


### 1. Set up Build Environment

This sets up the build environment, including Java and Maven versions, and the root directory of the build.


In [1]:
from subprocess import Popen, PIPE
import os
import shutil

Set the notebook directory. Note that this is system dependent.

In [2]:
NOTEBOOK_DIR = '/home/cybuilder/cytoscape-admin-scripts'

print(NOTEBOOK_DIR)

/home/cybuilder/cytoscape-admin-scripts


Set the Java environment. This is Java 11 for Cytoscape 3.8.0 and above.

Note that this is system dependent, and may need to be changed to reflect the system's configuration.

In [3]:
%env JAVA_HOME=/usr/lib/jvm/jre-11-openjdk

env: JAVA_HOME=/usr/lib/jvm/jre-11-openjdk


Set the MAVEN_HOME environment variable, as well as point the path to the correct Maven binaries. 

Note that this is system dependent and may need to be changed to reflect the system's configuration.

In [4]:
%env MAVEN_HOME=/opt/maven
%env PATH=/opt/apache-maven-3.6.0/bin/:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin

env: MAVEN_HOME=/opt/maven
env: PATH=/opt/apache-maven-3.6.0/bin/:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin


Prepare the target directory and clone the cytoscape git repo.

In [5]:
print('Changing to directory: ' + NOTEBOOK_DIR)
os.chdir(NOTEBOOK_DIR)

# Point to build location (the directory to clone parent cytoscape into)
BUILD_PARENT_DIR = os.path.join(os.getcwd(), 'install4j8_build')
if not os.path.exists(BUILD_PARENT_DIR):
    os.mkdir(BUILD_PARENT_DIR)
else:
    shutil.rmtree(BUILD_PARENT_DIR)
    os.mkdir(BUILD_PARENT_DIR)

os.chdir(BUILD_PARENT_DIR)
![[ -d cytoscape ]] || git clone https://github.com/cytoscape/cytoscape
CYTOSCAPE_ROOT_DIR = os.path.join(BUILD_PARENT_DIR, 'cytoscape')
CYTOSCAPE_DIR = os.path.join(CYTOSCAPE_ROOT_DIR, 'cytoscape')

def cd(directory=BUILD_PARENT_DIR, *subdirs):
    if subdirs:
        directory = os.path.join(directory, *subdirs)
    if os.getcwd() != directory:
        os.chdir(directory)

Changing to directory: /home/cybuilder/cytoscape-admin-scripts
Cloning into 'cytoscape'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 562 (delta 16), reused 19 (delta 7), pack-reused 529
Receiving objects: 100% (562/562), 766.15 KiB | 0 bytes/s, done.
Resolving deltas: 100% (269/269), done.


In [6]:
STARTING_BRANCH = 'develop'     # develop for major release, release/3.X.X for minor release

## 2. Pull the develop branch of Cytoscape

Note that to execute the checkout step here, you will need to have set up an SSH key on this machine that does not use password validation, or else parts of the build will be stall when they require input. 

In [7]:
cd(CYTOSCAPE_ROOT_DIR)
![[ -d cytoscape ]] || ./cy.sh init
cd(CYTOSCAPE_DIR)
!./cy.sh run-all "git checkout {STARTING_BRANCH}"

Target directory = 
Cytoscape project will be cloned to: /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape
Cloning into 'cytoscape'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 562 (delta 16), reused 19 (delta 7), pack-reused 529
Receiving objects: 100% (562/562), 766.15 KiB | 0 bytes/s, done.
Resolving deltas: 100% (269/269), done.
Cloning: parent (URI = git@github.com:cytoscape/cytoscape-parent.git)
Cloning into 'parent'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 1637 (delta 38), reused 33 (delta 9), pack-reused 1569
Receiving objects: 100% (1637/1637), 256.11 KiB | 0 bytes/s, done.
Resolving deltas: 100% (623/623), done.
~/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/parent ~/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape
Alre

## 2a. Reset

In [8]:
cd(CYTOSCAPE_DIR)
!./cy.sh run-all 'git clean -f -d'
!./cy.sh run-all 'git reset --hard'

------------------------------------------------------------------------
Executing command: git clean -f -d
--in .
------------------------------------------------------------------------
--in parent
------------------------------------------------------------------------
--in api
------------------------------------------------------------------------
--in impl
------------------------------------------------------------------------
--in support
------------------------------------------------------------------------
--in gui-distribution
------------------------------------------------------------------------
--in app-developer
------------------------------------------------------------------------
------------------------------------------------------------------------
Executing command: git reset --hard
--in .
HEAD is now at 8a10876 add aMatReader to core apps
------------------------------------------------------------------------
--in parent
HEAD is now at 8a407bd Update mockito

## 2. Verify status

In [9]:
cd(CYTOSCAPE_DIR)
!./cy.sh run-all 'git status'

------------------------------------------------------------------------
Executing command: git status
--in .
# On branch develop
nothing to commit, working directory clean
------------------------------------------------------------------------
--in parent
# On branch develop
nothing to commit, working directory clean
------------------------------------------------------------------------
--in api
# On branch develop
nothing to commit, working directory clean
------------------------------------------------------------------------
--in impl
# On branch develop
nothing to commit, working directory clean
------------------------------------------------------------------------
--in support
# On branch develop
nothing to commit, working directory clean
------------------------------------------------------------------------
--in gui-distribution
# On branch develop
nothing to commit, working directory clean
------------------------------------------------------------------------
--in app

## 3. Build Cytoscape and ensure no errors

This may take a while. Expect to build subrepos first before building from the root directory

Currently, you should expect two or so non-fatal javadoc-bundle-options related errors.

In [10]:
cd(CYTOSCAPE_DIR)
with open('build_output.txt', 'w') as outf:
    process = Popen('mvn clean install -Dmaven.test.skip=true'.split(' '), 
                stdout=outf,
                cwd=CYTOSCAPE_DIR)
    process.wait()

print("Showing ERROR lines in build...")
!cat build_output.txt | grep ERROR

Showing ERROR lines in build...
[ERROR] Error fetching link: /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/api/app-api/target/javadoc-bundle-options. Ignored it.
[ERROR] Error fetching link: /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/api/swing-app-api/target/javadoc-bundle-options. Ignored it.


## 4. Build Cytoscape installers

This requires Install4J to be configured on your machine and for the Install4j project file to point to the correct Mac signing file. The keystore password is set below (blank in this config), to be interpreted by the Install4j Maven plugin.

In [11]:
cd(CYTOSCAPE_DIR, 'gui-distribution', 'packaging')
%env MAC_KEYSTORE_PASSWORD=
!mvn clean install -U

env: MAC_KEYSTORE_PASSWORD=
[INFO] Scanning for projects...
[WARNING] 
[WARNING] Some problems were encountered while building the effective model for org.cytoscape.distribution:packaging:jar:3.9.0-SNAPSHOT
[WARNING] 'build.plugins.plugin.version' for org.sonatype.install4j:install4j-maven-plugin is missing. @ org.cytoscape.distribution:packaging:[unknown-version], /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/gui-distribution/packaging/pom.xml, line 104, column 15
[WARNING] 
[WARNING] It is highly recommended to fix these problems because they threaten the stability of your build.
[WARNING] 
[WARNING] For this reason, future Maven versions might no longer support building such malformed projects.
[WARNING] 
[INFO] 
[INFO] ----------------< org.cytoscape.distribution:packaging >----------------
[INFO] Building Cytoscape Release Packaging 3.9.0-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
Downloaded from central:

[INFO] No tests to run.
[INFO] 
[INFO] --- jacoco-maven-plugin:0.8.4:report (report) @ packaging ---
[INFO] Skipping JaCoCo execution due to missing execution data file.
[INFO] 
[INFO] --- maven-jar-plugin:2.4:jar (default-jar) @ packaging ---
[INFO] Building jar: /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/gui-distribution/packaging/target/packaging-3.9.0-SNAPSHOT.jar
[INFO] 
[INFO] >>> maven-source-plugin:3.0.1:jar (attach-sources) > generate-sources @ packaging >>>
[INFO] 
[INFO] --- jacoco-maven-plugin:0.8.4:prepare-agent (default) @ packaging ---
[INFO] argLine set to -javaagent:/home/cybuilder/.m2/repository/org/jacoco/org.jacoco.agent/0.8.4/org.jacoco.agent-0.8.4-runtime.jar=destfile=/home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/gui-distribution/packaging/target/jacoco.exec -Djdk.net.URLClassPath.disableClassPathURLCheck=true
[INFO] 
[INFO] <<< maven-source-plugin:3.0.1:jar (attach-sources) < generate-sources @ pack

[INFO]   Searching .i4j_external_228/framework/lib/openjfx/mac/javafx-swing-13-mac.jar for binaries
[INFO]   Searching .i4j_external_228/framework/lib/openjfx/mac/javafx-controls-13-mac.jar for binaries
[INFO]   Searching .i4j_external_228/framework/lib/openjfx/mac/javafx-media-13-mac.jar for binaries
[INFO]   Signing /libjfxmedia.dylib
[INFO]   Signing /libgstreamer-lite.dylib
[INFO]   Signing /libfxplugins.dylib
[INFO]   Signing /libglib-lite.dylib
[INFO]   Signing /libjfxmedia_avf.dylib
[INFO]   Searching .i4j_external_228/framework/lib/openjfx/mac/javafx-fxml-13-mac.jar for binaries
[INFO]   Searching .i4j_external_228/framework/lib/openjfx/mac/javafx-graphics-13-mac.jar for binaries
[INFO]   Signing /libprism_sw.dylib
[INFO]   Signing /libdecora_sse.dylib
[INFO]   Signing /libprism_es2.dylib
[INFO]   Signing /libprism_common.dylib
[INFO]   Signing /libjavafx_font.dylib
[INFO]   Signing /libglass.dylib
[INFO]   Signing /libjavafx_iio.dylib
[INFO]   Searching .i4j_external_228/frame

## 5. Copying Cytoscape installers to weekly download page

When completed installer executables can be found in `cytoscape/cytoscape/gui-distribution/packaging/target/media` and compressed builds for Linux and Windows can be found in `cytoscape/cytoscape/gui-distribution/assembly/target`

all of which should be copied to `/var/www/html/cytoscape-builds/Cytoscape-3.9.0/<DATE>`



In [12]:
!mkdir -p /var/www/html/cytoscape-builds/Cytoscape-3.9.0/$(date +%Y_%m_%d)
!rsync -av --progress /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/gui-distribution/packaging/target/media/* /var/www/html/cytoscape-builds/Cytoscape-3.9.0/$(date +%Y_%m_%d)
!rsync -av --progress /home/cybuilder/cytoscape-admin-scripts/install4j8_build/cytoscape/cytoscape/gui-distribution/assembly/target/*.{gz,zip} /var/www/html/cytoscape-builds/Cytoscape-3.9.0/$(date +%Y_%m_%d)


sending incremental file list
Cytoscape_3_9_0-SNAPSHOT_macos.dmg
    282,633,529 100%  211.05MB/s    0:00:01 (xfr#1, to-chk=7/8)
Cytoscape_3_9_0-SNAPSHOT_unix.sh
    280,485,554 100%  207.04MB/s    0:00:01 (xfr#2, to-chk=6/8)
Cytoscape_3_9_0-SNAPSHOT_windows_32bit.exe
    282,130,432 100%  238.95MB/s    0:00:01 (xfr#3, to-chk=5/8)
Cytoscape_3_9_0-SNAPSHOT_windows_64bit.exe
    282,237,952 100%  248.08MB/s    0:00:01 (xfr#4, to-chk=4/8)
md5sums
            290 100%    3.29kB/s    0:00:00 (xfr#5, to-chk=3/8)
output.txt
            825 100%    9.37kB/s    0:00:00 (xfr#6, to-chk=2/8)
sha256sums
            418 100%    4.75kB/s    0:00:00 (xfr#7, to-chk=1/8)
updates.xml
          1,738 100%   19.74kB/s    0:00:00 (xfr#8, to-chk=0/8)

sent 1,127,766,595 bytes  received 168 bytes  250,614,836.22 bytes/sec
total size is 1,127,490,738  speedup is 1.00
sending incremental file list
cytoscape-unix-3.9.0-SNAPSHOT.tar.gz
    277,707,968 100%  220.31MB/s    0:00:01 (xfr#1, to-chk=1/2)
cytoscape-wind

## 6. Notarizing on idekerlab-macmini

The install4j generated .dmg must be notarized to run on macOS 10.15 and above. The .dmg will be sent to idekerlab-macmini and submitted for notarization. An email will be sent to William Markuske when the notarization process is complete. As soon as the .dmg is notarized it will work and no further action is needed.

In [13]:
!rsync -av --progress /var/www/html/cytoscape-builds/Cytoscape-3.9.0/$(date +%Y_%m_%d)/*.dmg idekerlab@idekerlab-macmini.ucsd.edu:~/apps_to_notarize/
!ssh idekerlab@idekerlab-macmini.ucsd.edu '~/notarizedmg_jing.sh ~/apps_to_notarize/*.dmg Snapshot.$(date +%Y%m%d)'    

building file list ... 
1 file to consider
Cytoscape_3_9_0-SNAPSHOT_macos.dmg
    282,633,529 100%   79.60MB/s    0:00:03 (xfr#1, to-chk=0/1)

sent 282,668,184 bytes  received 46 bytes  62,815,162.22 bytes/sec
total size is 282,633,529  speedup is 1.00
No errors uploading '/Users/idekerlab/apps_to_notarize/Cytoscape_3_9_0-SNAPSHOT_macos.dmg'.
RequestUUID = 88a9ce56-8e9c-4696-ad61-f0abfbf81d37




You can check the status of the notarization by pasting in the `RequestUUID` value into the following command and running.

In [17]:
!ssh idekerlab@idekerlab-macmini.ucsd.edu '~/notarizestatus_jing.sh 88a9ce56-8e9c-4696-ad61-f0abfbf81d37'

No errors getting notarization info.

       Date: 2020-05-26 16:09:09 +0000
       Hash: 320a50f1226eebf9914242b319c9c323e95f69e2bab0e9775f3ea786ddc5ba20
RequestUUID: 88a9ce56-8e9c-4696-ad61-f0abfbf81d37
     Status: in progress

